# 🧠 Cofone — Complete Feature Demo

> **Simple, fast, yours.** Turn documents, websites and videos into a queryable knowledge base in a few lines of Python.

This notebook covers **every single feature** of the `cofone` library:

| # | Section |
|---|---|
| 0 | Setup & installation |
| 1 | Basic usage |
| 2 | Debug mode |
| 3 | Chunking modes (4 modes) |
| 4 | Retrieval — BM25 vs FAISS |
| 5 | FAISS persistence to disk |
| 6 | All 19 LLM providers |
| 7 | All 10 embedding providers |
| 8 | All source types (file, folder, PDF, web, Wikipedia, YouTube) |
| 9 | Multiple sources combined |
| 10 | Chat memory |
| 11 | Streaming |
| 12 | Structured output (Pydantic) |
| 13 | Custom tools |
| 14 | API key configuration (3 ways) |
| 15 | Power combos — advanced patterns |
| 16 | Full parameter reference |

---
## 0 · Setup

Install cofone with all optional extras:

In [ ]:
# Install everything
# pip install "cofone[all]"

# Or pick only what you need:
# pip install cofone              # base: txt, md, BM25, OpenRouter
# pip install "cofone[pdf]"       # adds PDF support (pypdf)
# pip install "cofone[faiss]"     # adds FAISS + sentence-transformers
# pip install "cofone[web]"       # adds Wikipedia + YouTube
# pip install "cofone[all]"       # everything above

print("Installation command: pip install 'cofone[all]'")

In [ ]:
# ── Standard imports used throughout this notebook ────────────────────────────
from pathlib import Path
from dotenv import load_dotenv
from cofone import RAG

# Load API keys from .env file in the project root
# The .env file should contain lines like:
#   OPENROUTER_API_KEY=sk-or-...
#   OPENAI_API_KEY=sk-...
#   ANTHROPIC_API_KEY=sk-ant-...
#   GEMINI_API_KEY=AI...
#   etc.
load_dotenv(Path().resolve().parent / '.env')

# Base path for local test files
BASE = Path().resolve()

print(f"BASE path: {BASE}")
print("Setup complete.")

---
## 1 · Basic usage

The simplest possible usage: load a file, ask a question, get an answer.

```
RAG()                        # create the RAG pipeline
  .add_source("file.txt")   # load a document
  .run("question")           # ask a question → returns a string
```

By default:
- LLM provider: **OpenRouter** (free tier, no credit card needed)
- Model: `arcee-ai/trinity-large-preview:free`
- Retrieval: **BM25** (keyword search, no extra deps)
- Chunking: **smart** (paragraph-aware)

In [ ]:
# Minimal example — everything at default
answer = RAG().add_source(BASE / 'note_ex.txt').run('Summarize')
print(answer)

In [ ]:
# Fluent one-liner — chain everything
print(RAG().add_source(BASE / 'note_ex.txt').run('What is the main topic of this document?'))

In [ ]:
# Ask multiple questions on the same source
src = RAG().add_source(BASE / 'note_ex.txt')

print(src.run('Give me a 2-sentence summary.'))
print()
print(src.run('What are the key points?'))
print()
print(src.run('What questions does this document raise?'))

---
## 2 · Debug mode

`.debug()` enables verbose logging. It prints:
- Provider and model being used
- Number of documents loaded
- Number of chunks created
- Which chunks were retrieved for the query (with text preview)
- Token estimate

Always useful during development to understand what's happening inside.

In [ ]:
# Debug mode — insert .debug() anywhere in the chain
answer = (
    RAG()
    .debug()                            # ← enables verbose logging
    .add_source(BASE / 'note_ex.txt')
    .run('Summarize')
)
print()
print('ANSWER:', answer)

In [ ]:
# Debug with FAISS — shows embedding model and vector dimensions
answer = (
    RAG(faiss=True)
    .debug()
    .add_source(BASE / 'note_ex.txt')
    .run('What is this about?')
)
print()
print('ANSWER:', answer)

---
## 3 · Chunking modes

Chunking = how text is split into pieces before indexing.
The chunk is the unit of retrieval — a smaller, focused piece of text sent to the LLM.

| Mode | Strategy | Best for |
|---|---|---|
| `smart` | paragraphs → sentences if >600 chars | articles, docs, Wikipedia |
| `paragraphs` | split on blank lines only | short self-contained paragraphs |
| `sentences` | split on `.!?`, group up to ~500 chars | legal, scientific, dense text |
| `fixed` | 500-char slices with 50-char overlap | raw data, logs, unstructured text |

In [ ]:
# smart (default) — paragraph-aware, breaks long paragraphs into sentences
# Algorithm:
#   1. Split by blank lines → paragraphs
#   2. If a paragraph > 600 chars → split into sentences
#   3. Group sentences until ~500 chars total, then start a new chunk
answer = RAG(chunk_mode='smart').add_source(BASE / 'note_ex.txt').run('What is this about?')
print('[smart]', answer)

In [ ]:
# paragraphs — splits ONLY on blank lines (\n\n)
# Each paragraph becomes exactly one chunk, regardless of its length.
# Warning: if a paragraph is very long, the chunk may exceed context window.
answer = RAG(chunk_mode='paragraphs').add_source(BASE / 'note_ex.txt').run('What is this about?')
print('[paragraphs]', answer)

In [ ]:
# sentences — splits on . ! ? boundaries, then groups to ~500 chars
# Finest granularity — every sentence is important.
# Good for: legal documents, scientific papers, technical manuals.
answer = RAG(chunk_mode='sentences').add_source(BASE / 'note_ex.txt').run('What is this about?')
print('[sentences]', answer)

In [ ]:
# fixed — exactly 500-char slices with 50-char overlap between consecutive chunks
# The overlap prevents losing context at boundaries.
# Good for: log files, CSV dumps, unstructured raw text.
answer = RAG(chunk_mode='fixed').add_source(BASE / 'note_ex.txt').run('What is this about?')
print('[fixed]', answer)

In [ ]:
# Compare chunk counts — see how many pieces each mode creates
print('Chunk counts for note_ex.txt:')
print('-' * 40)
for mode in ['smart', 'paragraphs', 'sentences', 'fixed']:
    rag = RAG(chunk_mode=mode).add_source(BASE / 'note_ex.txt')
    n = len(rag._retriever.chunks)
    print(f'  [{mode:12s}] → {n:3d} chunks')

In [ ]:
# Inspect the actual chunks created by each mode
print('First 3 chunks with mode=smart:')
print('=' * 60)
rag = RAG(chunk_mode='smart').add_source(BASE / 'note_ex.txt')
for i, chunk in enumerate(rag._retriever.chunks[:3]):
    print(f'--- Chunk {i+1} ({len(chunk)} chars) ---')
    print(chunk)
    print()

---
## 4 · Retrieval — BM25 vs FAISS

### BM25 (default)
Keyword-based. Ranks chunks by term frequency × inverse document frequency.
- ✅ Zero dependencies, instant, great for exact term matches
- ❌ "automobile" won't match a chunk about "cars"

### FAISS — semantic search
Vector-based. Converts chunks and query to dense vectors, retrieves by geometric proximity.
- ✅ Finds synonyms, paraphrases, conceptually related text
- ✅ "vehicle" matches "car", "automobile", "transportation"
- ❌ Requires `faiss-cpu` + embedding model (first run is slower)

Requires: `pip install "cofone[faiss]"`

In [ ]:
# BM25 — default, no extra setup needed
print('=== BM25 (keyword matching) ===')
answer_bm25 = (
    RAG()
    .debug()
    .add_source(BASE / 'docs_ex/')
    .run('Who is Leonardo?')
)
print('ANSWER:', answer_bm25)

In [ ]:
# FAISS — semantic search with local sentence-transformers embeddings
# Default embedding: all-MiniLM-L6-v2 (fast, English-focused, 384 dims)
# No API key needed — runs 100% locally
print('=== FAISS (semantic search, local embeddings) ===')
answer_faiss = (
    RAG(faiss=True)
    .debug()
    .add_source(BASE / 'docs_ex/')
    .run('Who is Leonardo?')
)
print('ANSWER:', answer_faiss)

In [ ]:
# FAISS with a multilingual embedding model
# paraphrase-multilingual-MiniLM-L12-v2 supports 50+ languages
# Use this when your documents are in Italian, French, Spanish, etc.
print('=== FAISS (multilingual embeddings) ===')
answer_multi = (
    RAG(
        faiss=True,
        embedding_provider='local',
        embedding_model='paraphrase-multilingual-MiniLM-L12-v2'
    )
    .debug()
    .add_source(BASE / 'docs_ex/')
    .run('Descrivi i dipinti di Leonardo')  # Italian query
)
print('ANSWER:', answer_multi)

In [ ]:
# Side-by-side comparison: BM25 vs FAISS on a semantic query
# A query using synonyms — BM25 may miss it, FAISS should find it
QUERY = 'What vehicles did the Renaissance genius design?'  # won't say 'Leonardo' directly

print('--- BM25 ---')
print(RAG().add_source(BASE / 'docs_ex/').run(QUERY))
print()
print('--- FAISS ---')
print(RAG(faiss=True).add_source(BASE / 'docs_ex/').run(QUERY))

---
## 5 · FAISS persistence (cache to disk)

Without `persist_path`, FAISS rebuilds the entire index every run.
For large document sets (PDFs, thousands of pages) this can take minutes.

`persist_path` saves the index to disk on first run, then loads it instantly on all subsequent runs.

**What gets saved:**
```
my_index/
  index.faiss      ← the vector index
  chunks.json      ← the original text chunks
  metadata.json    ← chunk count, embedding model, dimensions
```

In [ ]:
db = BASE / '.cofone_cache'

# FIRST RUN — computes embeddings and saves index to disk
# [cofone] index saved to .cofone_cache
print('=== First run (builds & saves) ===')
answer = (
    RAG(faiss=True, persist_path=db)
    .add_source(BASE / 'docs_ex/')
    .run('Who is Leonardo da Vinci?')
)
print(answer)

In [ ]:
# SECOND RUN — loads from disk instantly, no recomputation
# [cofone] index loaded from .cofone_cache (N chunks)
print('=== Second run (loads from disk, instant) ===')
answer = (
    RAG(faiss=True, persist_path=db)
    .add_source(BASE / 'docs_ex/')
    .run('What are his most famous inventions?')
)
print(answer)

In [ ]:
# Inspect what got saved
import os
if db.exists():
    print(f'Files in {db}:')
    for f in sorted(db.iterdir()):
        size_kb = f.stat().st_size / 1024
        print(f'  {f.name:25s}  {size_kb:.1f} KB')
else:
    print('Cache not found — run the cells above first.')

In [ ]:
# Persistence with a custom embedding provider (OpenAI)
# The metadata.json records which embedding model was used,
# so cofone can warn you if you try to load with a different model.

# RAG(
#     faiss=True,
#     embedding_provider='openai',
#     embedding_model='text-embedding-3-small',
#     persist_path='./openai_index'
# ).add_source('docs/').run('question')

---
## 6 · All 19 LLM providers

Cofone supports 19 LLM providers via the `model_provider` parameter.
The interface is identical for all of them — swap `model_provider` and `model`, rest stays the same.

### Auto-detection
If you pass only `model=` (without `model_provider=`), cofone auto-detects the provider:
- `gpt-*` → openai
- `gemini-*` → gemini
- `claude-*` → anthropic
- anything else → openrouter (default)

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# OPENROUTER — default provider
# Key: OPENROUTER_API_KEY
# 200+ models, free tier available, one key for everything
# https://openrouter.ai/keys
# ─────────────────────────────────────────────────────────────────────────────

# Default (free model, no explicit provider needed)
RAG().add_source(BASE / 'note_ex.txt').run('Summarize in one sentence')

In [ ]:
# OpenRouter — explicitly specify a free model
RAG(
    model_provider='openrouter',
    model='meta-llama/llama-3.3-70b-instruct:free'
).add_source(BASE / 'note_ex.txt').run('Summarize in one sentence')

In [ ]:
# OpenRouter — another free model (Gemini via OpenRouter)
RAG(
    model_provider='openrouter',
    model='google/gemini-2.0-flash-exp:free'
).add_source(BASE / 'note_ex.txt').run('Summarize in one sentence')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# OPENAI — Key: OPENAI_API_KEY
# Models: gpt-4o-mini, gpt-4o, gpt-4-turbo, o1-mini, o3-mini, etc.
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='openai',
#     model='gpt-4o-mini'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# Auto-detection: 'gpt-*' prefix → openai (no need for model_provider)
# RAG(model='gpt-4o-mini').add_source(BASE / 'note_ex.txt').run('Summarize')

# RAG(
#     model_provider='openai',
#     model='gpt-4o'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires OPENAI_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# ANTHROPIC — Key: ANTHROPIC_API_KEY
# Models: claude-3-5-haiku-20241022, claude-3-5-sonnet-20241022, claude-3-opus-20240229
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='anthropic',
#     model='claude-3-5-haiku-20241022'   # fastest & cheapest Claude
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# RAG(
#     model_provider='anthropic',
#     model='claude-3-5-sonnet-20241022'  # best quality Claude
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# Auto-detection: 'claude-*' prefix → anthropic
# RAG(model='claude-3-5-haiku-20241022').add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires ANTHROPIC_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# GEMINI — Key: GEMINI_API_KEY
# Models: gemini-2.0-flash, gemini-1.5-pro, gemini-1.5-flash
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='gemini',
#     model='gemini-2.0-flash'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# Auto-detection: 'gemini-*' prefix → gemini
# RAG(model='gemini-2.0-flash').add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires GEMINI_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# MISTRAL — Key: MISTRAL_API_KEY
# Models: mistral-large-latest, mistral-medium, codestral-latest
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='mistral',
#     model='mistral-large-latest'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires MISTRAL_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# GROQ — Key: GROQ_API_KEY
# Specialty: very fast inference (low latency)
# Models: llama-3.1-8b-instant, llama-3.3-70b-versatile, mixtral-8x7b-32768
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='groq',
#     model='llama-3.1-8b-instant'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# RAG(
#     model_provider='groq',
#     model='llama-3.3-70b-versatile'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires GROQ_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COHERE — Key: COHERE_API_KEY
# Specialty: RAG-optimized models, great multilingual support
# Models: command-r-plus, command-r, command
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='cohere',
#     model='command-r-plus'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires COHERE_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# DEEPSEEK — Key: DEEPSEEK_API_KEY
# Specialty: DeepSeek-R1 reasoning model (chain-of-thought)
# Models: deepseek-chat, deepseek-reasoner
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='deepseek',
#     model='deepseek-reasoner'
# ).add_source(BASE / 'note_ex.txt').run('Summarize step by step')

print('Uncomment to use — requires DEEPSEEK_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# xAI — Key: XAI_API_KEY
# Models: grok-2, grok-beta
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='xai',
#     model='grok-2'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires XAI_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# TOGETHER — Key: TOGETHER_API_KEY
# Specialty: wide selection of open-source models
# Models: meta-llama/Llama-3-70b-chat-hf, mistralai/Mixtral-8x7B-Instruct-v0.1
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='together',
#     model='meta-llama/Llama-3-70b-chat-hf'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires TOGETHER_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# PERPLEXITY — Key: PERPLEXITY_API_KEY
# Specialty: web-connected models (real-time information)
# Models: llama-3.1-sonar-large-128k-online
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='perplexity',
#     model='llama-3.1-sonar-large-128k-online'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires PERPLEXITY_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# FIREWORKS — Key: FIREWORKS_API_KEY
# Specialty: fast open-model inference
# Models: accounts/fireworks/models/llama-v3p1-70b-instruct
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='fireworks',
#     model='accounts/fireworks/models/llama-v3p1-70b-instruct'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires FIREWORKS_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CEREBRAS — Key: CEREBRAS_API_KEY
# Specialty: ultra-fast inference (dedicated silicon)
# Models: llama3.1-8b, llama3.1-70b
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='cerebras',
#     model='llama3.1-70b'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires CEREBRAS_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# NVIDIA NIM — Key: NVIDIA_API_KEY
# Specialty: NVIDIA-optimized models on cloud infrastructure
# Models: nvidia/llama-3.1-nemotron-70b-instruct
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='nvidia',
#     model='nvidia/llama-3.1-nemotron-70b-instruct'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires NVIDIA_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# DEEPINFRA — Key: DEEPINFRA_API_KEY
# Specialty: cheap open-model inference
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='deepinfra',
#     model='meta-llama/Meta-Llama-3.1-70B-Instruct'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires DEEPINFRA_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# ANYSCALE — Key: ANYSCALE_API_KEY
# Specialty: scalable inference with Ray-based infrastructure
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='anyscale',
#     model='meta-llama/Llama-3-70b-chat-hf'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires ANYSCALE_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# OLLAMA — local, no key, no internet needed
# Install: https://ollama.com
# Pull models: ollama pull llama3
# Ollama must be running: ollama serve
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='ollama',
#     model='llama3'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# RAG(
#     model_provider='ollama',
#     model='mistral'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# RAG(
#     model_provider='ollama',
#     model='phi3'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires Ollama running locally')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# LM STUDIO — local, no key, no internet needed
# Install: https://lmstudio.ai
# Start the local server in LM Studio, then:
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='lmstudio',
#     model='any-model-loaded-in-lmstudio'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires LM Studio with local server enabled')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# LLAMA.CPP — local, no key, no internet needed
# Install: pip install llama-cpp-python
# Download a .gguf model and start the server:
#   python -m llama_cpp.server --model model.gguf
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='llamacpp',
#     model='local'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires llama.cpp server running locally')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CUSTOM OPENAI-COMPATIBLE ENDPOINT
# Any server that speaks the OpenAI API format works via base_url=
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='openai',
#     model='my-custom-model',
#     base_url='http://my-server:8000/v1',
#     model_api_key='optional-key-if-needed'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — point base_url to any OpenAI-compatible server')

---
## 7 · All 10 embedding providers

Embeddings are only used when `faiss=True`. They transform text into vectors.
The LLM and embedding providers are **completely independent** — mix any combination.

| Provider | `embedding_provider=` | Needs key | Notes |
|---|---|---|---|
| sentence-transformers | `'local'` | No | Default. Runs offline. |
| OpenAI | `'openai'` | Yes | High quality, fast |
| Gemini | `'gemini'` | Yes | text-embedding-004 |
| Cohere | `'cohere'` | Yes | Multilingual specialist |
| Mistral | `'mistral'` | Yes | mistral-embed |
| Voyage | `'voyage'` | Yes | Top retrieval quality |
| Jina | `'jina'` | Yes | jina-embeddings-v3 |
| NVIDIA | `'nvidia'` | Yes | nv-embed-v2 |
| Together | `'together'` | Yes | BGE, UAE models |
| Ollama | `'ollama'` | No | Local, nomic-embed-text |

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# LOCAL (sentence-transformers) — default, no key, runs offline
# Default model: all-MiniLM-L6-v2
#   - Fast, 384 dimensions, English-focused
#   - Downloaded automatically on first use (~22 MB)
# ─────────────────────────────────────────────────────────────────────────────

RAG(
    faiss=True,
    embedding_provider='local',         # can also omit this — it's the default
    embedding_model='all-MiniLM-L6-v2'  # can also omit this — it's the default
).add_source(BASE / 'note_ex.txt').run('Summarize')

In [ ]:
# LOCAL — larger model, better quality (but slower)
# all-mpnet-base-v2: 768 dims, higher quality retrieval
RAG(
    faiss=True,
    embedding_provider='local',
    embedding_model='all-mpnet-base-v2'
).add_source(BASE / 'note_ex.txt').run('Summarize')

In [ ]:
# LOCAL — multilingual model (50+ languages)
# Use this for Italian, French, Spanish, German, etc. documents
RAG(
    faiss=True,
    embedding_provider='local',
    embedding_model='paraphrase-multilingual-MiniLM-L12-v2'
).add_source(BASE / 'note_ex.txt').run('Di cosa parla questo testo?')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# OPENAI embeddings — Key: OPENAI_API_KEY
# text-embedding-3-small: fast, cheap, 1536 dims
# text-embedding-3-large: best quality, 3072 dims
# text-embedding-ada-002: legacy, 1536 dims
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='openai',
#     embedding_model='text-embedding-3-small'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

# RAG(
#     faiss=True,
#     embedding_provider='openai',
#     embedding_model='text-embedding-3-large'  # higher quality, higher cost
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires OPENAI_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# GEMINI embeddings — Key: GEMINI_API_KEY
# text-embedding-004: 768 dims
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='gemini',
#     embedding_model='text-embedding-004'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires GEMINI_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COHERE embeddings — Key: COHERE_API_KEY
# Extra: pip install cohere
# Specialty: multilingual, great for non-English documents
# Models: embed-multilingual-v3.0, embed-english-v3.0
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='cohere',
#     embedding_model='embed-multilingual-v3.0'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires COHERE_API_KEY + pip install cohere')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# MISTRAL embeddings — Key: MISTRAL_API_KEY
# Models: mistral-embed (1024 dims)
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='mistral',
#     embedding_model='mistral-embed'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires MISTRAL_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# VOYAGE — Key: VOYAGE_API_KEY
# Extra: pip install voyageai
# Specialty: consistently top-ranked retrieval quality in benchmarks
# Models: voyage-3, voyage-3-lite, voyage-code-3
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='voyage',
#     embedding_model='voyage-3'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires VOYAGE_API_KEY + pip install voyageai')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# JINA — Key: JINA_API_KEY
# Models: jina-embeddings-v3 (supports multilingual, 8192 token context)
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='jina',
#     embedding_model='jina-embeddings-v3'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires JINA_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# NVIDIA embeddings — Key: NVIDIA_API_KEY
# Models: nvidia/nv-embed-v2
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='nvidia',
#     embedding_model='nvidia/nv-embed-v2'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires NVIDIA_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# TOGETHER embeddings — Key: TOGETHER_API_KEY
# Models: BAAI/bge-large-en-v1.5, WhereIsAI/UAE-Large-V1
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='together',
#     embedding_model='BAAI/bge-large-en-v1.5'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires TOGETHER_API_KEY')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# OLLAMA embeddings — local, no key, no internet needed
# Default model: nomic-embed-text
# Pull: ollama pull nomic-embed-text
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     faiss=True,
#     embedding_provider='ollama',
#     embedding_model='nomic-embed-text'
# ).add_source(BASE / 'note_ex.txt').run('Summarize')

print('Uncomment to use — requires Ollama running locally + nomic-embed-text pulled')

---
## 8 · All source types

All sources are loaded via `.add_source()`. Cofone auto-detects the type from the path or URL.

| Source | Auto-detected from |
|---|---|
| `.txt` / `.md` file | file extension |
| Folder | path ends with `/` or is a directory |
| PDF | `.pdf` extension |
| Wikipedia | `wikipedia.org` in URL |
| YouTube | `youtube.com` or `youtu.be` in URL |
| Generic web URL | any other URL |

In [ ]:
# ── Single .txt file ──────────────────────────────────────────────────────────
answer = RAG().add_source(BASE / 'note_ex.txt').run('Summarize')
print('[.txt]', answer)

In [ ]:
# ── Folder — loads all .txt, .md, .pdf files recursively ─────────────────────
# Every file in docs_ex/ and all its subfolders gets loaded into a single index
answer = RAG().add_source(BASE / 'docs_ex/').run('What are all the topics covered in these documents?')
print('[folder]', answer)

In [ ]:
# ── .md (Markdown) file ───────────────────────────────────────────────────────
# Treated same as .txt — plain text extraction
answer = RAG().add_source(BASE / 'README.md').run('What does this project do?')
print('[.md]', answer)

In [ ]:
# ── PDF — requires pip install "cofone[pdf]" ──────────────────────────────────
# Uses pypdf internally for text extraction

# RAG().add_source('report.pdf').run('What are the key findings?')
# RAG().add_source('contracts/').run('What are the payment terms?')  # folder of PDFs

print('Uncomment to use — requires pip install "cofone[pdf]"')

In [ ]:
# ── Wikipedia — requires pip install "cofone[web]" ────────────────────────────
# Auto-detected from wikipedia.org URL
# Returns clean article text (no markup, no nav, no sidebars)
# Supports any language: en.wikipedia, it.wikipedia, fr.wikipedia, etc.

answer = (
    RAG()
    .debug()
    .add_source('https://en.wikipedia.org/wiki/Artificial_intelligence')
    .run('What is the definition of AI and when was the field founded?')
)
print(answer)

In [ ]:
# Wikipedia in Italian
answer = (
    RAG()
    .add_source('https://it.wikipedia.org/wiki/Leonardo_da_Vinci')
    .run('Dove nacque Leonardo da Vinci?')
)
print(answer)

In [ ]:
# ── Generic web URL ───────────────────────────────────────────────────────────
# Fetches the page HTML, strips nav/footer/scripts, keeps only visible text

answer = (
    RAG()
    .add_source('https://docs.python.org/3/library/pathlib.html')
    .run('How do I create a Path object and check if a file exists?')
)
print(answer)

In [ ]:
# ── YouTube transcript — requires pip install "cofone[web]" ───────────────────
# Fetches official subtitles/transcript (auto-generated captions also work)
# Language priority: Italian first, then English

answer = (
    RAG()
    .debug()
    .add_source('https://www.youtube.com/watch?v=jNQXAC9IVRw')  # "Me at the zoo" (first YouTube video)
    .run('What is this video about?')
)
print(answer)

In [ ]:
# YouTube — short URL format also works
# answer = RAG().add_source('https://youtu.be/jNQXAC9IVRw').run('Summarize')
print('Both youtube.com and youtu.be short URLs are supported.')

---
## 9 · Multiple sources combined

Chain as many `.add_source()` calls as you want.
All documents are merged into a single index. Retrieval searches across all of them.

In [ ]:
# Combine a local file + a folder
answer = (
    RAG()
    .add_source(BASE / 'note_ex.txt')
    .add_source(BASE / 'docs_ex/')
    .run('What do you know about Cofone and Leonardo?')
)
print(answer)

In [ ]:
# Combine local files + Wikipedia + YouTube
answer = (
    RAG()
    .add_source(BASE / 'note_ex.txt')
    .add_source('https://en.wikipedia.org/wiki/Machine_learning')
    .run('Give me a complete overview combining all sources')
)
print(answer)

In [ ]:
# Debug multiple sources — see total chunk count from all sources
rag = (
    RAG()
    .debug()
    .add_source(BASE / 'note_ex.txt')
    .add_source(BASE / 'docs_ex/')
)
print('Total chunks:', len(rag._retriever.chunks))

---
## 10 · Chat memory

Two ways to enable memory:
1. Use `.chat()` instead of `.run()` — automatically enables memory
2. Pass `memory=True` to the constructor — keeps memory across `.run()` calls

Memory injects the full conversation history into every prompt:
```
Conversation so far:
User: Who is Leonardo da Vinci?
Assistant: He was a Renaissance polymath...

User: When was he born?
Assistant: He was born on April 15, 1452...

Context:
[retrieved chunks]

User: What are his most famous works?
```

In [ ]:
# Method 1: use .chat() — automatic memory, simplest API
bot = RAG().add_source(BASE / 'docs_ex/')

r1 = bot.chat('Who is Leonardo da Vinci?')
print('Q1:', r1)
print()

r2 = bot.chat('When was he born?')          # 'he' refers to Leonardo — context is preserved
print('Q2:', r2)
print()

r3 = bot.chat('What are his most famous paintings?')
print('Q3:', r3)
print()

r4 = bot.chat('Did he have any unfinished works?')  # 'he' still = Leonardo
print('Q4:', r4)

In [ ]:
# Reset memory — next call starts fresh, no previous context
bot.reset_memory()

r_fresh = bot.chat('What are we talking about?')  # no context — should say it doesn't know
print('After reset:', r_fresh)

In [ ]:
# Method 2: memory=True constructor flag — keeps history across .run() calls
# Useful when you want memory but prefer the .run() interface
bot2 = RAG(memory=True).add_source(BASE / 'docs_ex/')

print(bot2.run('Who is Leonardo?'))
print()
print(bot2.run('What did he invent?'))   # has context from previous call
print()
print(bot2.run('How old was he when he started painting?'))

In [ ]:
# Chat memory + FAISS — full combo
bot3 = RAG(faiss=True).add_source(BASE / 'docs_ex/')

print(bot3.chat('Summarize the documents briefly.'))
print()
print(bot3.chat('Which part was most interesting to you?'))  # conversational follow-up
print()
print(bot3.chat('Tell me more about that.'))

---
## 11 · System prompt

The system prompt is prepended to **every** prompt and tells the LLM who it is and how to behave.

Default built-in prompt:
```
You are a helpful assistant. Answer the user's question using ONLY the information
provided in the context below. If the answer is not in the context, say you don't know.
Be concise, accurate, and respond in the same language as the user's question.
```
Override with `system_prompt=` to set a custom role, tone, language or format.

In [ ]:
# Default — no system_prompt needed, built-in prompt is used
print(RAG().add_source(BASE / 'note_ex.txt').run('Summarize'))

In [ ]:
# Custom role — art historian
print(
    RAG(
        system_prompt='You are an expert art historian. Answer formally and with precision.'
    ).add_source(BASE / 'docs_ex/').run('Tell me about Leonardo')
)

In [ ]:
# Force Italian output regardless of query language
print(
    RAG(
        system_prompt='Rispondi sempre in italiano, in modo chiaro e sintetico.'
    ).add_source(BASE / 'docs_ex/').run('What is the main topic?')
)

In [ ]:
# Control output format — always bullet points
print(
    RAG(
        system_prompt='Answer always with a bullet-point list. Maximum 5 points. No prose.'
    ).add_source(BASE / 'docs_ex/').run('Summarize the key facts about Leonardo')
)

In [ ]:
# system_prompt + memory + streaming — full combo
bot = RAG(
    system_prompt='You are a friendly tutor. Explain concepts simply and use examples.',
    memory=True,
    max_history=5,
).add_source(BASE / 'docs_ex/')

for token in bot.stream('Who is Leonardo da Vinci? Explain simply.'):
    print(token, end='', flush=True)
print()
print()
for token in bot.stream('What is he most famous for?'):
    print(token, end='', flush=True)
print()

---
## 12 · Streaming

`.stream()` is a generator that yields string tokens one by one as they arrive from the LLM.
No waiting for the full response — tokens print to the screen in real time.

Works with all providers that support streaming (OpenRouter, OpenAI, Anthropic, Gemini, Mistral, Groq, Cohere, Ollama, ...).

In [ ]:
# Basic streaming
rag = RAG().add_source(BASE / 'docs_ex/')

print('Streaming response:')
print('-' * 50)
for token in rag.stream("Tell me about Leonardo's inventions"):
    print(token, end='', flush=True)
print()  # newline after streaming ends

In [ ]:
# Streaming with debug — shows chunk info before streaming starts
rag2 = RAG().debug().add_source(BASE / 'note_ex.txt')

print('Streaming with debug:')
print('-' * 50)
for token in rag2.stream('Describe this document in detail'):
    print(token, end='', flush=True)
print()

In [ ]:
# Collect streamed tokens into a full string (if you need it later)
tokens = []
for token in RAG().add_source(BASE / 'note_ex.txt').stream('Summarize'):
    print(token, end='', flush=True)
    tokens.append(token)
print()

full_response = ''.join(tokens)
print(f'\nTotal characters received: {len(full_response)}')

In [ ]:
# Streaming with FAISS — semantic search + real-time output
rag3 = RAG(faiss=True).add_source(BASE / 'docs_ex/')

for token in rag3.stream('What is the most interesting thing about Leonardo?'):
    print(token, end='', flush=True)
print()

---
## 12 · Structured output (Pydantic)

Pass a Pydantic `BaseModel` as `schema=` to `.run()`.
Instead of a string, you get back a **validated Python object**.

How it works internally:
1. The schema is converted to JSON Schema and appended to the prompt
2. The LLM is instructed to respond only with a JSON object
3. `response_format={'type': 'json_object'}` is passed where supported
4. The response is parsed and validated with `schema.model_validate()`

In [ ]:
from pydantic import BaseModel
from typing import List, Optional

# Simple schema — extract person data
class Person(BaseModel):
    name: str
    birth_year: int
    nationality: str
    most_famous_work: str

data = RAG().add_source(BASE / 'docs_ex/').run(
    'Extract data about Leonardo da Vinci',
    schema=Person
)

print(f'Type:             {type(data)}')
print(f'Name:             {data.name}')
print(f'Birth year:       {data.birth_year}')
print(f'Nationality:      {data.nationality}')
print(f'Most famous work: {data.most_famous_work}')

In [ ]:
# Schema with List fields
class ArtistProfile(BaseModel):
    name: str
    birth_year: int
    death_year: Optional[int]
    nationality: str
    main_disciplines: List[str]       # list of strings
    notable_works: List[str]
    historical_period: str

profile = RAG().add_source(BASE / 'docs_ex/').run(
    'Build a complete profile of Leonardo da Vinci',
    schema=ArtistProfile
)

import json
print(json.dumps(profile.model_dump(), indent=2))

In [ ]:
# Nested schema — schema with nested objects
class Work(BaseModel):
    title: str
    year: Optional[int]
    category: str  # painting, sculpture, drawing, etc.
    description: str

class ArtistWorks(BaseModel):
    artist_name: str
    total_known_works: int
    featured_works: List[Work]

result = RAG().add_source(BASE / 'docs_ex/').run(
    'List the most famous works of Leonardo da Vinci with details',
    schema=ArtistWorks
)

print(f'Artist: {result.artist_name}')
print(f'Total known works: {result.total_known_works}')
print()
for w in result.featured_works:
    print(f'  [{w.year}] {w.title} ({w.category})')
    print(f'        {w.description[:80]}...')

In [ ]:
# Schema from Wikipedia source
class AISummary(BaseModel):
    definition: str
    main_subfields: List[str]
    biggest_challenge: str
    year_field_founded: Optional[int]
    key_researchers: List[str]

result = (
    RAG()
    .add_source('https://en.wikipedia.org/wiki/Artificial_intelligence')
    .run('Extract structured info about AI', schema=AISummary)
)
print(result.model_dump_json(indent=2))

In [ ]:
# Classification schema — classify the document
from typing import Literal

class DocumentClassification(BaseModel):
    document_type: Literal['article', 'manual', 'biography', 'report', 'fiction', 'other']
    language: str
    main_topics: List[str]
    sentiment: Literal['positive', 'neutral', 'negative']
    approximate_reading_time_minutes: int
    suitable_for_ages: str  # e.g. "all ages", "adults only", "children"

classification = RAG().add_source(BASE / 'note_ex.txt').run(
    'Classify and analyze this document',
    schema=DocumentClassification
)
print(f'Type:          {classification.document_type}')
print(f'Language:      {classification.language}')
print(f'Topics:        {classification.main_topics}')
print(f'Sentiment:     {classification.sentiment}')
print(f'Read time:     ~{classification.approximate_reading_time_minutes} min')

---
## 13 · Custom tools

Attach Python functions with `.add_tool()`. The LLM can call them during reasoning.
Tools are described to the LLM alongside the retrieved context — it decides when to call them.

Requirements: the function must have a **docstring** (it's used as the tool description for the LLM).

In [ ]:
# Basic tool: calculator
def calculate(expression: str) -> str:
    """Evaluate a Python math expression and return the numeric result."""
    try:
        result = eval(expression, {'__builtins__': {}}, {})
        return f'Result: {result}'
    except Exception as e:
        return f'Error: {e}'

answer = (
    RAG()
    .add_tool(calculate)
    .add_source(BASE / 'note_ex.txt')
    .run('Summarize the document and tell me what 144 divided by 12 is')
)
print(answer)

In [ ]:
# Multiple tools attached to the same RAG
def calculate(expression: str) -> str:
    """Evaluate a Python math expression and return the numeric result."""
    try:
        return f'Result: {eval(expression)}'
    except Exception as e:
        return f'Error: {e}'

def word_count(text: str) -> str:
    """Count the number of words in a text string."""
    return f'Word count: {len(text.split())}'

def reverse_text(text: str) -> str:
    """Reverse the characters in a string."""
    return text[::-1]

def current_date() -> str:
    """Return the current date and time in ISO format."""
    from datetime import datetime
    return datetime.now().isoformat()

answer = (
    RAG()
    .add_tool(calculate)
    .add_tool(word_count)
    .add_tool(reverse_text)
    .add_tool(current_date)
    .add_source(BASE / 'note_ex.txt')
    .run('What is today\'s date? Also, what is 2 to the power of 10?')
)
print(answer)

In [ ]:
# Tool that queries an external service (example: local filesystem)
def list_files(folder: str) -> str:
    """List all files in a given folder path and return their names."""
    p = Path(folder)
    if not p.exists():
        return f'Folder {folder} does not exist.'
    files = [f.name for f in p.iterdir() if f.is_file()]
    return f'Files in {folder}: {files}'

answer = (
    RAG()
    .add_tool(list_files)
    .add_source(BASE / 'note_ex.txt')
    .run(f'List the files in {str(BASE)}')
)
print(answer)

---
## 14 · API key configuration (3 ways)

Keys are resolved in this priority order:
1. **Direct parameter** (`model_api_key=` or `embedding_api_key=`) — highest priority
2. **`.env` file** — recommended for development
3. **System environment variable** — for CI/CD, production

In [ ]:
# ── Way 1: direct parameter — highest priority, overrides .env ────────────────
# RAG(model_api_key='sk-or-...').add_source('note.txt').run('Summarize')

# For embedding key separately (when faiss=True with API embedding):
# RAG(
#     faiss=True,
#     embedding_provider='openai',
#     embedding_model='text-embedding-3-small',
#     embedding_api_key='sk-...',
# ).add_source('note.txt').run('Summarize')

# Both LLM and embedding keys directly:
# RAG(
#     model_provider='openai',
#     model='gpt-4o-mini',
#     model_api_key='sk-...',
#     faiss=True,
#     embedding_provider='openai',
#     embedding_model='text-embedding-3-small',
#     embedding_api_key='sk-...',
# ).add_source('note.txt').run('Summarize')

print('Way 1: direct parameter — see commented code above')

In [ ]:
# ── Way 2: .env file ──────────────────────────────────────────────────────────
# Create a file named .env in your project root with these contents:

env_template = """
# LLM providers
OPENROUTER_API_KEY=sk-or-...
OPENAI_API_KEY=sk-...
ANTHROPIC_API_KEY=sk-ant-...
GEMINI_API_KEY=AI...
MISTRAL_API_KEY=...
GROQ_API_KEY=gsk_...
COHERE_API_KEY=...
DEEPSEEK_API_KEY=...
XAI_API_KEY=xai-...
TOGETHER_API_KEY=...
FIREWORKS_API_KEY=...
PERPLEXITY_API_KEY=pplx-...
CEREBRAS_API_KEY=...
NVIDIA_API_KEY=nvapi-...
DEEPINFRA_API_KEY=...
ANYSCALE_API_KEY=...

# Embedding-only providers
VOYAGE_API_KEY=...
JINA_API_KEY=...
"""
print('Way 2: .env template:')
print(env_template)

In [ ]:
# Then at the top of every script/notebook:
from dotenv import load_dotenv
load_dotenv()  # reads .env from current dir and all parent dirs

# Or specify path explicitly:
load_dotenv(Path().resolve().parent / '.env')

print('Keys loaded from .env (if file exists).')

In [ ]:
# ── Way 3: system environment variable ───────────────────────────────────────
# Set before running the script. Persists for the session.

# PowerShell (Windows):
# $env:OPENROUTER_API_KEY = "sk-or-..."

# bash / zsh (Linux / macOS):
# export OPENROUTER_API_KEY="sk-or-..."

# Jupyter: set in the kernel environment or use %env magic:
# %env OPENROUTER_API_KEY=sk-or-...

# Verify what's currently set:
import os
providers = [
    'OPENROUTER_API_KEY', 'OPENAI_API_KEY', 'ANTHROPIC_API_KEY',
    'GEMINI_API_KEY', 'GROQ_API_KEY', 'MISTRAL_API_KEY'
]
print('Current API keys status:')
for key in providers:
    val = os.environ.get(key, '')
    status = '✓ set' if val else '✗ not set'
    print(f'  {key:<25} {status}')

---
## 15 · Power combos — advanced patterns

Real-world combinations that use multiple features together.

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 1: Fully local RAG — no internet, no API keys, 100% private
# Ollama LLM + Ollama embeddings + FAISS + local files
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='ollama',
#     model='llama3',
#     faiss=True,
#     embedding_provider='ollama',
#     embedding_model='nomic-embed-text'
# ).add_source('confidential_docs/').run('Summarize')

print('Combo 1: fully local — see commented code above')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 2: Max quality — best LLM + best embeddings + persistent index
# GPT-4o + OpenAI embeddings + FAISS persisted to disk
# ─────────────────────────────────────────────────────────────────────────────

# RAG(
#     model_provider='openai',
#     model='gpt-4o',
#     faiss=True,
#     embedding_provider='openai',
#     embedding_model='text-embedding-3-large',
#     persist_path='./premium_index'
# ).add_source('docs/').run('Deep analysis of all documents')

print('Combo 2: max quality — see commented code above')

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 3: Chat bot with memory + streaming + FAISS
# Conversational agent that searches semantically and streams in real time
# ─────────────────────────────────────────────────────────────────────────────

bot = RAG(faiss=True, memory=True).add_source(BASE / 'docs_ex/')

questions = [
    'Who is the main subject of these documents?',
    'What were his most important contributions?',
    'Which of those contributions do you find most impressive and why?'
]

for q in questions:
    print(f'\n> {q}')
    print('-' * 50)
    for token in bot.stream(q):
        print(token, end='', flush=True)
    print()

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 4: Structured extraction pipeline
# Load Wikipedia + extract structured data + serialize to JSON
# ─────────────────────────────────────────────────────────────────────────────

import json
from pydantic import BaseModel
from typing import List, Optional

class TechProfile(BaseModel):
    full_name: str
    birth_year: Optional[int]
    nationality: str
    companies_founded: List[str]
    key_inventions: List[str]
    net_worth_usd_billions: Optional[float]
    current_role: str

profile = (
    RAG(faiss=True)
    .add_source('https://en.wikipedia.org/wiki/Elon_Musk')
    .run('Extract a complete profile of Elon Musk', schema=TechProfile)
)

print(json.dumps(profile.model_dump(), indent=2))

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 5: Multi-source research assistant with tools
# Combines docs + Wikipedia + custom tools + structured output
# ─────────────────────────────────────────────────────────────────────────────

from datetime import datetime

def today() -> str:
    """Return today's date in YYYY-MM-DD format."""
    return datetime.now().strftime('%Y-%m-%d')

def count_words(text: str) -> str:
    """Count words in a text and return the count."""
    return str(len(text.split()))

answer = (
    RAG(chunk_mode='smart')
    .add_tool(today)
    .add_tool(count_words)
    .add_source(BASE / 'note_ex.txt')
    .add_source('https://en.wikipedia.org/wiki/Python_(programming_language)')
    .run('What is today\'s date? And summarize what Python is based on the Wikipedia article.')
)
print(answer)

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 6: Fast free combo — OpenRouter + BM25 + multiple free models compared
# ─────────────────────────────────────────────────────────────────────────────

FREE_MODELS = [
    'arcee-ai/trinity-large-preview:free',
    'meta-llama/llama-3.3-70b-instruct:free',
    'google/gemini-2.0-flash-exp:free',
]

QUERY = 'Summarize this document in exactly 2 sentences.'

print(f'Query: {QUERY}\n')
for model in FREE_MODELS:
    answer = (
        RAG(model_provider='openrouter', model=model)
        .add_source(BASE / 'note_ex.txt')
        .run(QUERY)
    )
    print(f'[{model}]')
    print(answer)
    print()

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# COMBO 7: Debug all chunking modes + both retrieval strategies
# Benchmark how different configs affect chunk count and answer quality
# ─────────────────────────────────────────────────────────────────────────────

QUERY = 'Who invented the telephone?'

configs = [
    {'name': 'BM25 + smart',      'faiss': False, 'chunk_mode': 'smart'},
    {'name': 'BM25 + sentences',  'faiss': False, 'chunk_mode': 'sentences'},
    {'name': 'FAISS + smart',     'faiss': True,  'chunk_mode': 'smart'},
    {'name': 'FAISS + sentences', 'faiss': True,  'chunk_mode': 'sentences'},
]

for cfg in configs:
    rag = RAG(faiss=cfg['faiss'], chunk_mode=cfg['chunk_mode'])
    rag.add_source(BASE / 'note_ex.txt')
    n_chunks = len(rag._retriever.chunks)
    answer = rag.run(QUERY)
    print(f'[{cfg["name"]}] — {n_chunks} chunks')
    print(f'  → {answer[:100]}...')
    print()

---
## 16 · Full parameter reference

Complete documentation of every parameter and method.

### `RAG()` constructor parameters

| Parameter | Type | Default | Description |
|---|---|---|---|
| `model_provider` | `str` | `'openrouter'` | LLM provider. Options: `'openrouter'`, `'openai'`, `'anthropic'`, `'gemini'`, `'mistral'`, `'groq'`, `'cohere'`, `'deepseek'`, `'xai'`, `'together'`, `'perplexity'`, `'fireworks'`, `'cerebras'`, `'nvidia'`, `'deepinfra'`, `'anyscale'`, `'ollama'`, `'lmstudio'`, `'llamacpp'` |
| `model` | `str` | provider default | LLM model identifier string |
| `model_api_key` | `str` | `None` | LLM API key — overrides `.env` |
| `base_url` | `str` | provider default | Custom LLM endpoint (any OpenAI-compatible API) |
| `faiss` | `bool` | `False` | Use FAISS semantic search. If `False`, uses BM25 (default) |
| `embedding_provider` | `str` | `'local'` | Embedding provider. Options: `'local'`, `'openai'`, `'gemini'`, `'cohere'`, `'mistral'`, `'voyage'`, `'jina'`, `'nvidia'`, `'together'`, `'ollama'` |
| `embedding_model` | `str` | `'all-MiniLM-L6-v2'` | Embedding model identifier |
| `embedding_api_key` | `str` | `None` | Embedding API key — overrides `.env` |
| `chunk_mode` | `str` | `'smart'` | Chunking strategy. Options: `'smart'`, `'paragraphs'`, `'sentences'`, `'fixed'` |
| `persist_path` | `str\|Path` | `None` | Folder to save/load FAISS index. Only used when `faiss=True` |
| `memory` | `bool` | `False` | Enable conversation history. Equivalent to always using `.chat()` || `system_prompt` | `str` | `None` | Custom instructions for the LLM. `None` = built-in default |


### Methods

| Method | Returns | Description |
|---|---|---|
| `.add_source(path_or_url)` | `self` | Load file, folder, URL, Wikipedia, YouTube. Chainable — call multiple times. |
| `.add_tool(fn)` | `self` | Attach a Python function. Function must have a docstring. |
| `.debug()` | `self` | Enable verbose logging: provider, model, chunks, retrieval preview. |
| `.run(query, schema=None)` | `str` or Pydantic model | Single stateless query. If `schema=` is passed, returns a validated Pydantic object. |
| `.chat(query)` | `str` | Stateful query with automatic memory. Context accumulates across calls. |
| `.stream(query)` | `Generator[str, None, None]` | Generator yielding string tokens one by one as they arrive from the LLM. |
| `.reset_memory()` | `self` | Clear all conversation history. Next call starts fresh. |

### Key resolution order

```
model_api_key= parameter  ← highest priority
  ↓ (if not set)
.env file  (e.g. OPENROUTER_API_KEY=...)
  ↓ (if not set)
system environment variable
  ↓ (if not set)
no key needed (local providers: ollama, lmstudio, llamacpp)
```

### Source auto-detection rules

```python
add_source("file.txt")                              # → plain text file
add_source("README.md")                             # → markdown (treated as plain text)
add_source("report.pdf")                            # → PDF (requires cofone[pdf])
add_source("docs/")                                 # → folder, loads all .txt .md .pdf
add_source("https://en.wikipedia.org/wiki/...")    # → Wikipedia article
add_source("https://it.wikipedia.org/wiki/...")    # → Wikipedia in Italian
add_source("https://www.youtube.com/watch?v=...")  # → YouTube transcript
add_source("https://youtu.be/...")                  # → YouTube short URL
add_source("https://any-other-url.com")             # → generic web page
```

In [ ]:
# Quick syntax reference — all in one cell
print("""
╔══════════════════════════════════════════════════════════════════╗
║                    COFONE QUICK REFERENCE                        ║
╠══════════════════════════════════════════════════════════════════╣
║  MINIMAL USAGE                                                   ║
║  RAG().add_source("file.txt").run("question")                    ║
╠══════════════════════════════════════════════════════════════════╣
║  LLM PROVIDERS (19)                                              ║
║  openrouter* openai anthropic gemini mistral groq cohere         ║
║  deepseek xai together perplexity fireworks cerebras nvidia      ║
║  deepinfra anyscale  |  ollama* lmstudio llamacpp (local)        ║
╠══════════════════════════════════════════════════════════════════╣
║  EMBEDDING PROVIDERS (10)                                        ║
║  local* openai gemini cohere mistral voyage jina nvidia together ║
║  ollama (local)                                                  ║
╠══════════════════════════════════════════════════════════════════╣
║  CHUNK MODES (4)                                                 ║
║  smart* paragraphs sentences fixed                               ║
╠══════════════════════════════════════════════════════════════════╣
║  SOURCES                                                         ║
║  file.txt / README.md / report.pdf / folder/ / wikipedia URL     ║
║  youtube URL / any web URL / chain multiple .add_source() calls  ║
╠══════════════════════════════════════════════════════════════════╣
║  METHODS                                                         ║
║  .run(q)           → str     (stateless)                         ║
║  .run(q, schema=M) → Pydantic (structured output)                ║
║  .chat(q)          → str     (with memory)                       ║
║  .stream(q)        → tokens  (real-time)                         ║
║  .reset_memory()   → self    (clear history)                     ║
║  .debug()          → self    (verbose logging)                   ║
║  .add_tool(fn)     → self    (attach Python function)            ║
╠══════════════════════════════════════════════════════════════════╣
║  * = default value                                               ║
╚══════════════════════════════════════════════════════════════════╝
""")